In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
class Blocking_System:
    def __init__(self, m: int, service, arrival, 
                 arrival_method: str = 'poisson', service_method: str = 'exp'): 
        self.clock=0.0                      #simulation clock
        self.num_service_units=m            #system with m service units
        self.arrival_mtd=arrival_method     #arrival distribution
        self.service_mtd=service_method     #service distribution
        self.param_service=service          #parameter for service dist.
        self.param_arrival=arrival          #parameter for arrival dist.
        self.num_arrivals=0                 #total number of arrivals
        self.t_arrival=self.gen_arrival_time()   #time of next arrival
        self.t_departures=np.ones(m)*100000. #departure times for each service unit (100.000 as infinite)
        self.dep_sums=np.zeros((m,), dtype=int) #Sum of service time
        self.states=np.zeros((m,), dtype=int) #current states
        self.num_of_departures=np.zeros((m,), dtype=int) #number of customers served
        self.lost_customers=0               #customers who left without service
        self.num_in_system=0                #customers in the system


    def time_adv(self):                                                       
        t_departure=min(self.t_departures)
        idx = list(self.t_departures).index(t_departure)
        if self.t_arrival<t_departure:
            self.clock=self.t_arrival
            self.arrival()
        else:
            self.clock=t_departure
            self.departure(idx)


    def arrival(self):              
        self.num_arrivals += 1
        self.num_in_system += 1

        accepted = False
        for idx in range(self.num_service_units):
            if self.states[idx]==0:
                accepted = True
                dep=self.gen_service_time()
                self.dep_sums[idx] += dep
                self.t_departures[idx]=self.clock + dep
                self.states[idx]=1
                break

        self.t_arrival=self.clock+self.gen_arrival_time()
        if not accepted:
            self.lost_customers += 1


    def departure(self, idx: int):
        self.num_of_departures[idx] += 1
        self.t_departures[idx]=100000. # (100.000 as infinite)
        self.states[idx]=0                  


    def gen_arrival_time(self):         #function to generate arrival times 
        if self.arrival_mtd=='erlang':
            return (np.random.gamma(self.param_arrival)) # Erlang distribution (using Gamma with shape=int)
        elif self.arrival_mtd=='hyperexp': # Hyper Exponential distribution p1 = 0.8, λ1 = 0.8333, p2 = 0.2, λ2 = 5.0
            if np.random.uniform() <= 0.8: #p1
                return (np.random.exponential(scale=1./0.833)) #λ1
            else: #p2
                return (np.random.exponential(scale=1./5.)) #λ2
        else:
            return (np.random.poisson()) # Poisson distribution

    
    def gen_service_time(self):         #function to generate service time
        if self.service_mtd=='constant':
            return self.param_service
        if self.service_mtd=='pareto': # Pareto distribution
            return (np.random.pareto(self.param_service))
        if self.service_mtd=='normal':
            return (np.random.normal(loc=self.param_service)) # Normal Distribution
        else:
            return (np.random.exponential()) # Exponential distribution (lamda=1)
    

In [3]:
results = pd.DataFrame([],columns=['run','mean','count','std','CI low limit','CI high limit','CI range'])

**Write a discrete event simulation program for a blocking system, i.e. a system with m service units and no waiting room. The offered traffic A is the product of the mean arrival rate and the mean service time.**

**1. The arrival process is modelled as a Poisson process. Report the fraction of blocked customers, and a confidence interval for this fraction. Choose the service time distribution as exponential. Parameters: m = 10, mean service time = 8 time units, mean time between customers = 1 time unit (corresponding to an offered traffic of 8 erlang), 10 x 10.000 customers.**

In [4]:
m = 10
mu_service = 8
mu_arrival = 1
s=Blocking_System(m, mu_service, mu_arrival)
df=pd.DataFrame(columns=['Fraction of blocked customers','Average interarrival time','Total Customers','Blocked Customers'])

for i in range(10):
    np.random.seed(i)
    s.__init__(m, mu_service, mu_arrival)
    while s.num_in_system <= 10000 :
        s.time_adv() 
    a=pd.Series([s.lost_customers/s.num_arrivals,s.clock/s.num_arrivals,s.num_arrivals,s.lost_customers],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df  

,Fraction of blocked customers,Average interarrival time,Total Customers,Blocked Customers
0,0.0005,0.993201,10001.0,5.0
1,0.0001,0.991801,10001.0,1.0
2,0.0002,0.996800,10001.0,2.0
3,0.0000,1.006099,10001.0,0.0
4,0.0000,0.986401,10001.0,0.0
5,0.0000,1.019698,10001.0,0.0
6,0.0002,1.004200,10001.0,2.0
7,0.0001,0.985001,10001.0,1.0
8,0.0004,0.989101,10001.0,4.0
9,0.0001,0.999000,10001.0,1.0


In [5]:
stats = df['Fraction of blocked customers'].agg(['mean', 'count', 'std'])
ci95_hi = []
ci95_lo = []
m = stats.loc['mean']
c = stats.loc['count']
s = stats.loc['std']
ci95_hi.append(m + 1.95*s/math.sqrt(c))
ci95_lo.append(m - 1.95*s/math.sqrt(c))

stats['CI low limit'] = ci95_lo
stats['CI high limit'] = ci95_hi
stats['CI range'] = ci95_hi[0] - ci95_lo[0]
print(stats)

stats['run'] = 'Q1 Poisson'
results = results.append(stats,ignore_index=True)

mean                             0.00016
count                               10.0
std                             0.000171
CI low limit     [5.438206253347918e-05]
CI high limit    [0.0002655859406662009]
CI range                        0.000211
Name: Fraction of blocked customers, dtype: object


**2. The arrival process is modelled as a renewal process using the same parameters as in Part 1 when possible. Report the fraction of blocked customers, and a confidence interval for this fraction for at least the following two cases**

- **(a) Experiment with Erlang distributed inter arrival times The Erlang distribution should have a mean of 1**

In [6]:
m = 10
mu_service = 8
mu_arrival = 1
s=Blocking_System(m, mu_service, mu_arrival, arrival_method='erlang')
df=pd.DataFrame(columns=['Fraction of blocked customers','Average interarrival time','Total Customers','Blocked Customers'])

for i in range(10):
    np.random.seed(i)
    s.__init__(m, mu_service, mu_arrival, arrival_method='erlang')
    while s.num_in_system <= 10000 :
        s.time_adv() 
    a=pd.Series([s.lost_customers/s.num_arrivals,s.clock/s.num_arrivals,s.num_arrivals,s.lost_customers],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df  

,Fraction of blocked customers,Average interarrival time,Total Customers,Blocked Customers
0,0.0,0.989077,10001.0,0.0
1,0.0,0.995992,10001.0,0.0
2,0.0,0.999760,10001.0,0.0
3,0.0,1.003446,10001.0,0.0
4,0.0,1.006480,10001.0,0.0
5,0.0,1.022823,10001.0,0.0
6,0.0,0.997917,10001.0,0.0
7,0.0,0.990758,10001.0,0.0
8,0.0,0.994180,10001.0,0.0
9,0.0,0.993435,10001.0,0.0


In [7]:
stats = df['Fraction of blocked customers'].agg(['mean', 'count', 'std'])
ci95_hi = []
ci95_lo = []
m = stats.loc['mean']
c = stats.loc['count']
s = stats.loc['std']
ci95_hi.append(m + 1.95*s/math.sqrt(c))
ci95_lo.append(m - 1.95*s/math.sqrt(c))

stats['CI low limit'] = ci95_lo
stats['CI high limit'] = ci95_hi
stats['CI range'] = ci95_hi[0] - ci95_lo[0]
print(stats)

stats['run'] = 'Q2 Erlang for arrival'
results = results.append(stats,ignore_index=True)

mean               0.0
count             10.0
std                0.0
CI low limit     [0.0]
CI high limit    [0.0]
CI range           0.0
Name: Fraction of blocked customers, dtype: object


- **(b) hyper exponential inter arrival times. The parameters for the hyper exponential distribution should be p1 = 0.8, λ1 = 0.8333, p2 = 0.2, λ2 = 5.0.**

In [8]:
m = 10
mu_service = 8
mu_arrival = 1
s=Blocking_System(m, mu_service, mu_arrival, arrival_method='hyperexp')
df=pd.DataFrame(columns=['Fraction of blocked customers','Average interarrival time','Total Customers','Blocked Customers'])

for i in range(10):
    np.random.seed(i)
    s.__init__(m, mu_service, mu_arrival, arrival_method='hyperexp')
    while s.num_in_system <= 10000 :
        s.time_adv() 
    a=pd.Series([s.lost_customers/s.num_arrivals,s.clock/s.num_arrivals,s.num_arrivals,s.lost_customers],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df    

,Fraction of blocked customers,Average interarrival time,Total Customers,Blocked Customers
0,0.0,0.988889,10001.0,0.0
1,0.0,0.990259,10001.0,0.0
2,0.0,1.014987,10001.0,0.0
3,0.0,1.006684,10001.0,0.0
4,0.0,1.009461,10001.0,0.0
5,0.0,1.008191,10001.0,0.0
6,0.0,1.002576,10001.0,0.0
7,0.0,0.999442,10001.0,0.0
8,0.0,0.978068,10001.0,0.0
9,0.0,1.012754,10001.0,0.0


In [9]:
stats = df['Fraction of blocked customers'].agg(['mean', 'count', 'std'])
ci95_hi = []
ci95_lo = []
m = stats.loc['mean']
c = stats.loc['count']
s = stats.loc['std']
ci95_hi.append(m + 1.95*s/math.sqrt(c))
ci95_lo.append(m - 1.95*s/math.sqrt(c))

stats['CI low limit'] = ci95_lo
stats['CI high limit'] = ci95_hi
stats['CI range'] = ci95_hi[0] - ci95_lo[0]
print(stats)

stats['run'] = 'Q2 Hyperexponential for arrival'
results = results.append(stats,ignore_index=True)

mean               0.0
count             10.0
std                0.0
CI low limit     [0.0]
CI high limit    [0.0]
CI range           0.0
Name: Fraction of blocked customers, dtype: object


**3. The arrival process is again a Poisson process like in Part 1. Experiment with different service time distributions with the same mean service time and m as in Part 1 and Part 2.**

- **(a) Constant service time**

In [10]:
m = 10
mu_service = 8.
mu_arrival = 1
s=Blocking_System(m, mu_service, mu_arrival, service_method='constant')
df=pd.DataFrame(columns=['Fraction of blocked customers','Average interarrival time','Total Customers','Blocked Customers'])

for i in range(10):
    np.random.seed(i)
    s.__init__(m, mu_service, mu_arrival, service_method='constant')
    while s.num_in_system <= 10000 :
        s.time_adv() 
    a=pd.Series([s.lost_customers/s.num_arrivals,s.clock/s.num_arrivals,s.num_arrivals,s.lost_customers],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df    

,Fraction of blocked customers,Average interarrival time,Total Customers,Blocked Customers
0,0.108189,0.990401,10001.0,1082.0
1,0.108889,1.006199,10001.0,1089.0
2,0.106989,0.992801,10001.0,1070.0
3,0.112589,0.993501,10001.0,1126.0
4,0.106389,1.000600,10001.0,1064.0
5,0.100290,1.020798,10001.0,1003.0
6,0.102890,1.006699,10001.0,1029.0
7,0.104890,0.997900,10001.0,1049.0
8,0.110089,0.985201,10001.0,1101.0
9,0.104190,1.002800,10001.0,1042.0


In [11]:
stats = df['Fraction of blocked customers'].agg(['mean', 'count', 'std'])
ci95_hi = []
ci95_lo = []
m = stats.loc['mean']
c = stats.loc['count']
s = stats.loc['std']
ci95_hi.append(m + 1.95*s/math.sqrt(c))
ci95_lo.append(m - 1.95*s/math.sqrt(c))

stats['CI low limit'] = ci95_lo
stats['CI high limit'] = ci95_hi
stats['CI range'] = ci95_hi[0] - ci95_lo[0]
print(stats)

stats['run'] = 'Q3 Constant for service'
results = results.append(stats,ignore_index=True)

mean                          0.106539
count                             10.0
std                           0.003625
CI low limit     [0.10430386251083941]
CI high limit    [0.10877482961994751]
CI range                      0.004471
Name: Fraction of blocked customers, dtype: object


- **(b) Pareto distributed service times with at least k = 1.05 and k = 2.05.**

In [12]:
m = 10
k = 1.05
mu_arrival = 1
s=Blocking_System(m, k, mu_arrival, service_method='pareto')
df=pd.DataFrame(columns=['Fraction of blocked customers','Average interarrival time','Total Customers','Blocked Customers'])

for i in range(10):
    np.random.seed(i)
    s.__init__(m, k, mu_arrival, service_method='pareto')
    while s.num_in_system <= 10000 :
        s.time_adv() 
    a=pd.Series([s.lost_customers/s.num_arrivals,s.clock/s.num_arrivals,s.num_arrivals,s.lost_customers],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df    

,Fraction of blocked customers,Average interarrival time,Total Customers,Blocked Customers
0,0.070993,0.991401,10001.0,710.0
1,0.051495,0.996700,10001.0,515.0
2,0.064294,1.000500,10001.0,643.0
3,0.093891,1.009399,10001.0,939.0
4,0.133187,0.984302,10001.0,1332.0
5,0.110889,1.019198,10001.0,1109.0
6,0.103290,1.006199,10001.0,1033.0
7,0.056194,0.985001,10001.0,562.0
8,0.029897,0.989301,10001.0,299.0
9,0.048595,0.997400,10001.0,486.0


In [13]:
stats = df['Fraction of blocked customers'].agg(['mean', 'count', 'std'])
ci95_hi = []
ci95_lo = []
m = stats.loc['mean']
c = stats.loc['count']
s = stats.loc['std']
ci95_hi.append(m + 1.95*s/math.sqrt(c))
ci95_lo.append(m - 1.95*s/math.sqrt(c))

stats['CI low limit'] = ci95_lo
stats['CI high limit'] = ci95_hi
stats['CI range'] = ci95_hi[0] - ci95_lo[0]
print(stats)

stats['run'] = 'Q3 Pareto k=1.05 for service'
results = results.append(stats,ignore_index=True)

mean                          0.076272
count                             10.0
std                           0.032631
CI low limit     [0.05615069985495928]
CI high limit    [0.09639404567048816]
CI range                      0.040243
Name: Fraction of blocked customers, dtype: object


In [14]:
m = 10
k = 2.05
mu_arrival = 1
s=Blocking_System(m, k, mu_arrival, service_method='pareto')
df=pd.DataFrame(columns=['Fraction of blocked customers','Average interarrival time','Total Customers','Blocked Customers'])

for i in range(10):
    np.random.seed(i)
    s.__init__(m, k, mu_arrival, service_method='pareto')
    while s.num_in_system <= 10000 :
        s.time_adv() 
    a=pd.Series([s.lost_customers/s.num_arrivals,s.clock/s.num_arrivals,s.num_arrivals,s.lost_customers],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df  

,Fraction of blocked customers,Average interarrival time,Total Customers,Blocked Customers
0,0.0004,0.993101,10001.0,4.0
1,0.0003,0.991701,10001.0,3.0
2,0.0002,0.996800,10001.0,2.0
3,0.0000,1.006099,10001.0,0.0
4,0.0000,0.986401,10001.0,0.0
5,0.0000,1.019698,10001.0,0.0
6,0.0002,1.004200,10001.0,2.0
7,0.0001,0.985001,10001.0,1.0
8,0.0003,0.989001,10001.0,3.0
9,0.0003,0.999000,10001.0,3.0


In [15]:
stats = df['Fraction of blocked customers'].agg(['mean', 'count', 'std'])
ci95_hi = []
ci95_lo = []
m = stats.loc['mean']
c = stats.loc['count']
s = stats.loc['std']
ci95_hi.append(m + 1.95*s/math.sqrt(c))
ci95_lo.append(m - 1.95*s/math.sqrt(c))

stats['CI low limit'] = ci95_lo
stats['CI high limit'] = ci95_hi
stats['CI range'] = ci95_hi[0] - ci95_lo[0]
print(stats)

stats['run'] = 'Q3 Pareto k=2.05 for service'
results = results.append(stats,ignore_index=True)

mean                            0.00018
count                              10.0
std                            0.000148
CI low limit      [8.8991100889911e-05]
CI high limit    [0.000270972902709729]
CI range                       0.000182
Name: Fraction of blocked customers, dtype: object


- **(c) Choose one or two other distributions.**

In [16]:
m = 10
mu_service = 8.
mu_arrival = 1
s=Blocking_System(m, mu_service, mu_arrival, service_method='normal')
df=pd.DataFrame(columns=['Fraction of blocked customers','Average interarrival time','Total Customers','Blocked Customers'])

for i in range(10):
    np.random.seed(i)
    s.__init__(m, mu_service, mu_arrival, service_method='normal')
    while s.num_in_system <= 10000 :
        s.time_adv() 
    a=pd.Series([s.lost_customers/s.num_arrivals,s.clock/s.num_arrivals,s.num_arrivals,s.lost_customers],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df  

,Fraction of blocked customers,Average interarrival time,Total Customers,Blocked Customers
0,0.129587,0.993301,10001.0,1296.0
1,0.124788,1.002800,10001.0,1248.0
2,0.130487,0.994601,10001.0,1305.0
3,0.131087,0.997700,10001.0,1311.0
4,0.126887,0.994201,10001.0,1269.0
5,0.122488,1.006599,10001.0,1225.0
6,0.128587,1.006699,10001.0,1286.0
7,0.133687,0.984902,10001.0,1337.0
8,0.130687,0.995500,10001.0,1307.0
9,0.125187,1.013199,10001.0,1252.0


In [17]:
stats = df['Fraction of blocked customers'].agg(['mean', 'count', 'std'])
ci95_hi = []
ci95_lo = []
m = stats.loc['mean']
c = stats.loc['count']
s = stats.loc['std']
ci95_hi.append(m + 1.95*s/math.sqrt(c))
ci95_lo.append(m - 1.95*s/math.sqrt(c))

stats['CI low limit'] = ci95_lo
stats['CI high limit'] = ci95_hi
stats['CI range'] = ci95_hi[0] - ci95_lo[0]
print(stats)

stats['run'] = 'Q3 Normal for service'
results = results.append(stats,ignore_index=True)

mean                          0.128347
count                             10.0
std                           0.003445
CI low limit        [0.12622312040599]
CI high limit    [0.13047121016095328]
CI range                      0.004248
Name: Fraction of blocked customers, dtype: object


**4. Compare confidence intervals for Parts 1, 2, and 3 and try explain differences if any.**

In [18]:
results

,run,mean,count,std,CI low limit,CI high limit,CI range
0,Q1 Poisson,0.000160,10.0,0.000171,[5.438206253347918e-05],[0.0002655859406662009],0.000211
1,Q2 Erlang for arrival,0.000000,10.0,0.000000,[0.0],[0.0],0.000000
2,Q2 Hyperexponential for arrival,0.000000,10.0,0.000000,[0.0],[0.0],0.000000
3,Q3 Constant for service,0.106539,10.0,0.003625,[0.10430386251083941],[0.10877482961994751],0.004471
4,Q3 Pareto k=1.05 for service,0.076272,10.0,0.032631,[0.05615069985495928],[0.09639404567048816],0.040243
5,Q3 Pareto k=2.05 for service,0.000180,10.0,0.000148,[8.8991100889911e-05],[0.000270972902709729],0.000182
6,Q3 Normal for service,0.128347,10.0,0.003445,[0.12622312040599],[0.13047121016095328],0.004248


The last 3 columns from the table above show the CI limits for each of the performed runs.

For the first 3 simulations the arrival process is being modified, where only with the Poisson function some customers are being blocked. This is due to the shape of this distribution, having its higher probabilities for lower values of x (closer to x=0). Erlang and exponential functions generate higher values, which in this case it would mean more time between arrivals.

For the last 4 runs, corresponding to Q3, the service time is being modified. The function making a bigger impact on the performance is Pareto with k=1.05. Compared to Pareto with k=2.05, with k=1.05 the distribution is more flat and tends to give higher values (less close to 0) than with k=1.05. Higher values from these distributions are translated as higher service times for the customers, increasing the blocking probability and its CI interval. In addition to this, it is logic to have a very similar CI for the constant value and Normal distribution for a high number of simulated customers (since the Normal distribution will be represented almost by its mean, which in this scenario is equal to the constant value).